In [1]:
import numpy as np
import h5py

from ase.build import bulk
from ase.calculators.emt import EMT
from vibes.helpers.supercell import make_cubic_supercell
from vibes.structure.io import inform

from vibes.phonopy import wrapper as ph, workflow as wf
from vibes.phono3py import wrapper as ph3, workflow as wf3

from vibes.helpers.pickle import pread

AttributeDict([('displacement', 0.03), ('symprec', 1e-05), ('trigonal', False), ('is_diagonal', False), ('q_mesh', [11, 11, 11]), ('cutoff_pair_distance', 100.0), ('log_level', 2)])


In [2]:
atoms = bulk('Al', cubic=True)

In [3]:
sc2, smatrix2 = make_cubic_supercell(atoms, 32)
sc3, smatrix3 = make_cubic_supercell(atoms, 32)
inform(sc3)


Geometry info for:
  input geometry:    Al32
  Symmetry prec.:    1e-05
  Number of atoms:   32
  Species:           Al
  Periodicity:       [ True  True  True]
  Lattice:  
    [8.1 0.  0. ]
    [0.  8.1 0. ]
    [0.  0.  8.1]

  Spacegroup:          Fm-3m (225)
  Wyckoff positions:   32*a
  Equivalent atoms:    32*a
  Standard lattice:  
    [4.05 0.   0.  ]
    [0.   4.05 0.  ]
    [0.   0.   4.05]
  supercell_matrix: [2, 0, 0, 0, 2, 0, 0, 0, 2]

Cell lengths and angles [Å, °]:
  a, b, c:      8.1000      8.1000      8.1000
  α, β, γ:     90.0000     90.0000     90.0000
  Volume:     531.4410 Å**3


In [4]:
phonopy_settings = {
    'atoms': atoms,
    'supercell_matrix': smatrix3,
    'workdir': 'fc2',
}

phono3py_settings = {
    'atoms': atoms,
    'supercell_matrix': smatrix3,
    'cutoff_pair_distance': 3,
    'workdir': 'fc3',
}

In [5]:
calc = EMT()

In [6]:
wf.run_phonopy(calculator=calc, **phonopy_settings)

** Warning from file /home/knoop/local/vibes/vibes/phonopy/workflow.py, line 50, function bootstrap:
--> Settings do not contain phonopy instructions.

** Warning from file /home/knoop/local/vibes/vibes/templates/aims.py, line 40, function setup_aims:
--> No [control] section in /home/knoop/local/vibes/vibes.cfg, return calc=None, good luck!

* Message from file /home/knoop/local/vibes/vibes/helpers/socketio.py, line 20, function get_port:
--> Socketio not used with calculator emt

Start postprocess.
done.


In [7]:
wf3.run_phono3py(calculator=calc, **phono3py_settings)

** Warning from file /home/knoop/local/vibes/vibes/phonopy/workflow.py, line 50, function bootstrap:
--> Settings do not contain phono3py instructions.

** Warning from file /home/knoop/local/vibes/vibes/templates/aims.py, line 40, function setup_aims:
--> No [control] section in /home/knoop/local/vibes/vibes.cfg, return calc=None, good luck!

* Message from file /home/knoop/local/vibes/vibes/helpers/socketio.py, line 20, function get_port:
--> Socketio not used with calculator emt

Start postprocess.
Computing fc3[ 1, x, x ] using numpy.linalg.pinv with a displacement:
    [ 0.0300  0.0000  0.0000]
Expanding fc3
Cutting-off fc3 (cut-off distance: 3.000000)
Building atom mapping table...
Creating contracted fc3...
done.


In [8]:
phonon = pread('phonon.pick.gz')
phonon3 = pread('phonon3.pick.gz')

In [9]:
(phonon.get_force_constants() == phonon3.get_fc2()).all()

True

In [10]:
fc2 = phonon3.get_fc2()
fc3 = phonon3.get_fc3()

In [11]:
scs2 = phonon.get_supercells_with_displacements()
scs3 = phonon3.get_supercells_with_displacements()
print(f'Number of FC2 displacements: {len(scs2)}')
print(f'Number of FC3 displacements: {len(scs3)}')
print(f'Number of FC3 displacements that were created: {len([sc for sc in scs3 if sc])}')

Number of FC2 displacements: 1
Number of FC3 displacements: 49
Number of FC3 displacements that were created: 18


In [12]:
q_mesh = 3*[5]
phonon3_new = ph3.prepare_phono3py(**phono3py_settings,
                                   fc2=fc2,
                                   fc3=fc3,
                                   q_mesh=q_mesh,
                                   log_level=0)

In [13]:
phonon3_new.run_thermal_conductivity(write_kappa=True)

In [14]:
with h5py.File('kappa-m{}.hdf5'.format(''.join(map(str, q_mesh)))) as kappa:
    # print(kappa['frequency'][:])
    print(kappa['kappa'][:][np.where(kappa['temperature'][:] == 300)][0, :])
    # assert 5 < kappa['frequency'][:].max() < 10, kappa['frequency'][:].max()
    # assert 25 < kappa['kappa'][:][30][:3].mean() < 35

[ 4.95576995e+00  4.95576995e+00  4.95576995e+00 -1.11909370e-17
  0.00000000e+00 -6.50113003e-18]
